# Préparation de la base sirene

Pour plus tard

df = ppro_df.loc[ppro_df.cross_l1.isin(['86_86','86_68','84_86','86_87','86_88'])]  

In [19]:
import os

import pandas as pd 

In [20]:
# Declaration des constantes
DATA_DIR = '../data/external/'
ENTITIES_FILENAME = 'StockUniteLegale_utf8.zip'
ETAB_FILENAME = 'StockEtablissement_utf8.zip'
ACTIVITY_CODE = '86.'
NAF_REF_FILENAME = "naf86.xlsx"
PROCESSED_DIR = '../data/processed/'
PS_CITIES_FILENAME = 'PS_communes.csv'

## Préparation des entités

In [ ]:
%%time
entities_df = (pd.read_csv(os.path.join(DATA_DIR, ENTITIES_FILENAME), # Concaténation du répertoire et du nom de fichier
                           usecols=['siren','denominationunitelegale',
                                    'activiteprincipaleunitelegale', 'datecreationunitelegale',
                                    'denominationunitelegale', 'nomunitelegale', 'etatadministratifunitelegale',
                                   ],
                           compression='zip',
                           #nrows=100000 # limite à n lignes la lecture
                          )
             ).dropna(subset=['activiteprincipaleunitelegale']).query('etatadministratifunitelegale == "A"')

# Filtrage des professionnels de santé
#entities_df = entities_df[entities_df.activiteprincipaleunitelegale.str.startswith(ACTIVITY_CODE)] # 86.

<string>:2: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.


In [ ]:
entities_df.shape

In [ ]:
entities_df.denominationunitelegale.isnull().sum()

In [ ]:
entities_df.nomunitelegale.isnull().sum()

In [ ]:
entities_df.denominationunitelegale.isnull().sum() + entities_df.nomunitelegale.isnull().sum()

In [ ]:
entities_df.head()

## Preparaton des etablissements

In [ ]:
%%time
etab_df = (pd.read_csv(os.path.join(DATA_DIR, ETAB_FILENAME), # Concaténation du répertoire et du nom de fichier
                       usecols=['siren',
                                'activiteprincipaleetablissement', 'datecreationetablissement',
                                'enseigne1etablissement', 'denominationusuelleetablissement',
                                'etatadministratifetablissement',
                                'codepostaletablissement', 'codecommuneetablissement', 'libellecommuneetablissement',
                               ],
                           compression='zip',
                       dtype = {'codepostaletablissement' : str,'codecommuneetablissement' : str}
                           #nrows=1_000_000 # limite à n lignes la lecture
                          )
             ).dropna(subset=['activiteprincipaleetablissement']).query('etatadministratifetablissement == "A"')

# Filtrage des professionnels de santé
#etab_df = etab_df[etab_df.activiteprincipaleetablissement.str.startswith(ACTIVITY_CODE)] # 86.

In [ ]:
etab_df.head(10)

In [ ]:
etab_df.dtypes

In [ ]:
entities_df.dtypes

In [ ]:
process_df = etab_df.merge(entities_df, on= 'siren', how= 'left')

In [ ]:
process_df.shape

In [ ]:
process_df[process_df.activiteprincipaleunitelegale.isnull()].shape

In [ ]:
process_df[process_df.activiteprincipaleunitelegale.isnull()].shape

In [ ]:
etab_df.memory_usage(deep=True).sum() / (1024 * 1024) # En mégaoctet

In [ ]:
entities_df.memory_usage(deep=True).sum() / (1024 * 1024) # En mégaoctet

In [ ]:
pro_df = etab_df.merge(entities_df, on= 'siren', how= 'outer', indicator =True)

## Fusion entre les entreprises et les établissements

In [ ]:
pro_df.head()

In [ ]:
#%%dpaste
ppro_df = (pro_df.loc[pro_df.activiteprincipaleunitelegale.str.startswith("86.") |
                      pro_df.activiteprincipaleetablissement.str.startswith("86.")]
           .assign(etab_apel1 = lambda df : df.activiteprincipaleetablissement.str.slice(0,2))
           .assign(el_apel1 = lambda df : df.activiteprincipaleunitelegale.str.slice(0,2))
           .assign(cross_l1 = lambda df : df.el_apel1 + '_' + df.etab_apel1 )
           
           
)

In [ ]:
pd.crosstab(ppro_df.etab_apel1,ppro_df.el_apel1)

In [ ]:
ppro_df.cross_l1.value_counts()

86_68 signifie qu'il est bien médecin mais il est logé dans un batiment ayant une activité mobilière </p>
84_86 c'est une administration publique mais c'est un hopital donc medecin dan sun hopital </p>
86_87 activité d'hébergement medico-social (maison de santé...)

## Analyse des croisements NAF entités établissements

https://www.insee.fr/fr/information/2120875

In [ ]:
ppro_df.cross_l1.value_counts(normalize = True).head()

In [ ]:
pd.crosstab(ppro_df.query('cross_l1 == "86_86"').activiteprincipaleetablissement,
           ppro_df.query('cross_l1 == "86_86"').activiteprincipaleunitelegale)

In [ ]:
ppro_df['code_activite'] = ppro_df.apply(lambda x : x.activiteprincipaleetablissement
                                        if x.etab_apel1 == '86'
                                        else x.activiteprincipaleunitelegale, axis = 1
                                        )

In [ ]:
ppro_df.code_activite.value_counts()

quand 86_86 je prend etablissement
quand il y a 86 quelque chose on prend le 86 des 2 

## Préparation de la table des codes NAF

In [17]:
#naf_df = pd.read_clipboard()
naf_df= pd.read_excel(os.path.join(DATA_DIR, NAF_REF_FILENAME))


In [18]:
naf_df

,NAF,Libellé,Catégorie
1914,86.10Z,Activités hospitalières,hopital
1918,86.21Z,Activité des médecins généralistes,generaliste
1920,86.22A,Activités de radiodiagnostic et de radiothérapie,radiologue
1921,86.22B,Activités chirurgicales,chirurgien
1922,86.22C,Autres activités des médecins spécialistes,autre_specialiste
1924,86.23Z,Pratique dentaire,dentiste
1928,86.90A,Ambulances,ambulance
1929,86.90B,Laboratoires d'analyses médicales,analyse_medicale
1930,86.90C,Centres de collecte et banques d'organes,organe
1931,86.90D,Activités des infirmiers et des sages-femmes,infirmiers


## Création de la table des communes

Tout d'abord on crée une dataframe ayant le code commune en lignes et les spécialités en colonnes 

|Commune|Spé 1 | Spé 2| ...|
------|--------|-------|----|
58290 | medecin| hopital| ...|


In [ ]:
commune_df= (ppro_df
 .merge(naf_df, left_on = 'code_activite', right_on ='NAF', how = 'inner')
 [['codecommuneetablissement','Catégorie']]
 .dropna(subset = ['codecommuneetablissement'])
 .assign(count = 1 )
 .groupby(['codecommuneetablissement','Catégorie'])
 .sum()
 .reset_index()
 .pivot(index = 'codecommuneetablissement', columns = 'Catégorie', values = 'count')
 .fillna(0)
)

In [ ]:
commune_df.head()

Enregistrement de la table comme on va en avoir besoin pour le merge final. On peut mettre en csv, pickle (objet pandas enregistré tel qu'il est sur le disque, se relit plus vite qu'un csv), ou hdF (interessant pour les gros fichiers, lisible sur python, R...)

In [ ]:
commune_df.to_csv(os.path.join(PROCESSED_DIR,PS_CITIES_FILENAME),sep = ';', dtype = {'codecommuneetablissement': str})